# Using Neo4jGraphRagCapability with agents for GraphRAG Question & Answering

AG2 provides GraphRAG integration using agent capabilities. This is an example to integrate Neo4j (a Property/Knowledge Graph database).

````{=mdx}
:::info Requirements
llama-index dependencies, which is required to use Neo4j prpoerty graph

```bash
pip llama-index llama-index-graph-stores-neo4j
```


## Set Configuration and OpenAI API Key

By default, in order to use FalkorDB you need to have an OpenAI key in your environment variable `OPENAI_API_KEY`.

You can utilise an OAI_CONFIG_LIST file and extract the OpenAI API key and put it in the environment, as will be shown in the following cell.

Alternatively, you can load the environment variable yourself.

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

In [1]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST", file_location="../")

# Put the OpenAI API key into the environment
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


````{=mdx}
:::important
The default model for loading graph data and answering questions using Neo4j is OpenAI's GPT 3.5-turbo, and the default embedding model is OpenAI's text-embedding-3-small,  This can be changed by setting the `model`and 'embed-model' parameter on the Neo4jGraphQueryEngine.
:::
````

In [2]:
# This is needed to allow nested asyncio calls for Neo4j in Jupyter
import nest_asyncio

nest_asyncio.apply()

## Create a Knowledge Graph with Your Own Data

**Note:** You need to have a Neo4j database running. If you are running one in a Docker container, please ensure your Docker network is setup to allow access to it. 

In this example, the Neo4j endpoint is set to host="bolt://172.17.0.4" and port=7687, please adjust accordingly. For how to spin up a Neo4j with Docker, you can refer to [this](https://docs.llamaindex.ai/en/stable/examples/property_graph/property_graph_neo4j/#:~:text=stores%2Dneo4j-,Docker%20Setup,%C2%B6,-To%20launch%20Neo4j)

Below, we have some sample data from Paul Grahma's [essay](https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt).

We then initialise the database with that text document, creating the graph in Neo4j.

### A Simple Example

In this example, the graph schema is auto-generated. This allows you to load data without specifying the specific types of entities and relationships that will make up the database (however, this may not be optimal and not cost efficient). 
First, we create a Neo4j property graph with Paul Grahma's essay.

In [3]:
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.neo4j_graph_query_engine import Neo4jGraphQueryEngine
from autogen.agentchat.contrib.graph_rag.neo4j_graph_rag_capability import Neo4jGraphCapability

# Auto generate graph schema from unstructured data
input_path = "../test/agentchat/contrib/graph_rag/paul_graham_essay.txt"
input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

# Create FalkorGraphQueryEngine
query_engine = Neo4jGraphQueryEngine(
    username="neo4j",  # Change if you reset username
    password="password",  # Change if you reset password
    host="bolt://172.17.0.4",  # Change
    port=7687,  # if needed
    database="neo4j",  # Change if you want to store the graphh in your custom database
)

# Ingest data and initialize the database
query_engine.init_db(input_doc=input_documents)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "MATCH (n:`Chunk`)\nWITH collect(distinct substring(toString(coalesce(n.`doc_id`, '')), 0, 50)) AS `doc_id_values`,\n     collect(distinct substring(toString(coalesce(n.`document_id`, '')), 0, 50)) AS `document_id_values`,\n     collect(distinct substring(toString(coalesce(n.`creation_date`, '')), 0, 50)) AS `creation_date_values`,\n     collect(distinct substring(toString(coalesce(n.`_node_type`, '')), 0, 50)) AS `_node_type_values`,\n     collect(distinct substring(toString(coalesce(n.`file_type`, '')), 0, 50)) AS `file_type_values`,\n     collect(distinct substring(toString(coalesce(n.`last_modified_date`, '')), 0, 50)) AS `last_modified_date_values`,\n     collect(distinct substri

### Add capability to a ConversableAgent and query them

In [4]:
# Create a ConversableAgent (no LLM configuration)
graph_rag_agent = ConversableAgent(
    name="paul_graham_agent",
    human_input_mode="NEVER",
)

# Associate the capability with the agent
graph_rag_capability = Neo4jGraphCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

# Create a user proxy agent to converse with our RAG agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
)

user_proxy.initiate_chat(graph_rag_agent, message="What happened at Interleaf and Viaweb?")

user_proxy (to paul_graham_agent):

What happened at Interleaf and Viaweb?

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

At Interleaf, a scripting language inspired by Emacs was added, with the scripting language being a dialect of Lisp. At Viaweb, there was a code editor for users to define their own page styles, which involved editing Lisp expressions underneath. Additionally, Viaweb had to recruit an initial set of users privately before launching publicly to ensure they had decent-looking stores.

--------------------------------------------------------------------------------
user_proxy (to paul_graham_agent):

How does paul graham do at Interleaf

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

Paul Graham did not perform well at Interleaf. He mentioned that he was a bad employee, as he did not understand most of the software due to his lack

ChatResult(chat_id=None, chat_history=[{'content': 'What happened at Interleaf and Viaweb?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'At Interleaf, a scripting language inspired by Emacs was added, with the scripting language being a dialect of Lisp. At Viaweb, there was a code editor for users to define their own page styles, which involved editing Lisp expressions underneath. Additionally, Viaweb had to recruit an initial set of users privately before launching publicly to ensure they had decent-looking stores.', 'role': 'user', 'name': 'paul_graham_agent'}, {'content': 'How does paul graham do at Interleaf', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Paul Graham did not perform well at Interleaf. He mentioned that he was a bad employee, as he did not understand most of the software due to his lack of knowledge in C and his reluctance to learn it. Additionally, he admitted to being terribly irresponsible during his time at Interleaf.', 'role': 'user', 'n